In [ ]:
from google.colab import files
import os

print("Please upload your 'All_Auth_Names.txt' file.")
uploaded = files.upload()

# Verify the file was uploaded
if 'All_Auth_Names.txt' in uploaded:
    print("\n✅ 'All_Auth_Names.txt' uploaded successfully!")
else:
    print("\n❌ File not found. Please make sure the uploaded file is named exactly 'All_Auth_Names.txt'.")

In [ ]:
# ==============================================================================
# Step 1: Install and Import Libraries
# ==============================================================================
!pip install flask requests pandas openpyxl pyngrok -q
import requests
import json
import pandas as pd
import io
import warnings
from flask import Flask, request, render_template_string, send_file
from pyngrok import ngrok, conf
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from datetime import datetime


# Patch Colab's debug inspection to avoid errors with Flask's request context
import google.colab._debugpy_repr
import types

def patched_get_shape(obj):
    try:
        return getattr(obj, 'shape', None)
    except Exception:
        return None

google.colab._debugpy_repr.get_shape = types.MethodType(patched_get_shape, google.colab._debugpy_repr)


# --- IMPORTANT: NGROK SETUP ---
# 1. Get your free token from https://dashboard.ngrok.com/get-started/your-authtoken
# 2. Paste it below.
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN_HERE" # <--- PASTE YOUR TOKEN ONLY HERE

if NGROK_AUTH_TOKEN == "YOUR_NGROK_AUTH_TOKEN_HERE":
    print("❌ ERROR: Please replace 'YOUR_NGROK_AUTH_TOKEN_HERE' with your actual ngrok token.")
else:
    conf.get_default().auth_token = NGROK_AUTH_TOKEN
    print("✅ Ngrok token configured successfully.")

# ==============================================================================
# Step 2: Function to Load Authority Names
# ==============================================================================
def load_auth_names():
    auth_names = []
    try:
        with open('All_Auth_Names.txt', 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip().startswith("Name:"):
                    name = line.split("Name:")[1].split(", Link:")[0].strip()
                    auth_names.append(name)
        print(f"✅ Loaded {len(auth_names)} authority names for autocomplete.")
        return sorted(list(set(auth_names)))
    except FileNotFoundError:
        print("⚠️ 'All_Auth_Names.txt' not found. Autocomplete will be disabled.")
        return []
    except Exception as e:
        print(f"Error reading auth file: {e}")
        return []

# ==============================================================================
# Step 3: API search function (This version is confirmed correct)
# ==============================================================================
def search_tenders_by_org(organization_name):
    API_URL = "https://t247_api.tender247.com/apigateway/T247Tender/api/tender/search-tender"
    HEADERS = {
        'Accept': 'application/json', 'Content-Type': 'application/json',
        'Origin': 'https://www.tender247.com', 'Referer': 'https://www.tender247.com/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36'
    }
    payload = {
        "tab_id": 2, "tender_id": 0, "tender_number": "", "search_text": "", "refine_search_text": "",
        "tender_value_operator": 0, "tender_value_from": 0, "tender_value_to": 0, "publication_date_from": "",
        "publication_date_to": "", "closing_date_from": "", "closing_date_to": "", "search_by_location": False,
        "statezone_ids": "", "city_ids": "", "state_ids": "", "organization_ids": 0,
        "organization_name": organization_name, "sort_by": 1, "sort_type": 2, "page_no": 1, "record_per_page": 500,
        "keyword_id": 0, "mfa": "", "nameof_website": "", "tender_typeid": 0, "is_tender_doc_uploaded": False,
        "exact_search": False, "exact_search_text": False, "search_by_split_word": False, "product_id": 0,
        "organization_type_id": 0, "sub_industry_id": 0, "search_by": 0, "guest_user_id": 0, "msme": 0,
        "startup": 0, "gem": 0, "quantity": "", "quantityOperator": 0, "is_ai_summary": False, "boq": False
    }

    try:
        warnings.filterwarnings('ignore', category=InsecureRequestWarning)
        response = requests.post(API_URL, headers=HEADERS, json=payload, timeout=45, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        return {'Success': False, 'Message': f'A network error occurred: {e}'}
    except json.JSONDecodeError:
        return {'Success': False, 'Message': 'Failed to decode API response.', 'RawResponse': response.text}

# ==============================================================================
# Step 4: Define the HTML Template
# ==============================================================================
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Tender-Search</title>
    <style>
        body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif; background-color: #f0f2f5; margin: 0; padding: 20px; }
        .container { max-width: 950px; margin: 20px auto; background: #fff; padding: 30px; border-radius: 10px; box-shadow: 0 6px 12px rgba(0,0,0,0.1); }
        h1 { text-align: center; color: #1d3557; }
        form { display: flex; gap: 10px; margin-bottom: 25px; }
        input[type="text"] { flex-grow: 1; padding: 12px; border: 1px solid #ccc; border-radius: 5px; font-size: 16px; }
        button { padding: 12px 25px; background-color: #007bff; color: white; border: none; border-radius: 5px; cursor: pointer; font-size: 16px; }
        .results-header { display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap; gap: 10px; margin: 20px 0 10px 0; }
        .export-button { background-color: #28a745; }
        .alert { padding: 15px; margin-bottom: 20px; border: 1px solid transparent; border-radius: 4px; }
        .alert-info { color: #0c5460; background-color: #d1ecf1; border-color: #bee5eb; }
        .alert-danger { color: #721c24; background-color: #f8d7da; border-color: #f5c6cb; }
        table { width: 100%; border-collapse: collapse; margin-top: 20px; table-layout: fixed; }
        th, td { padding: 12px 15px; border: 1px solid #ddd; text-align: left; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; max-width: 220px; }
        td:hover { white-space: normal; overflow: visible; background: #fff; z-index: 2; position: relative; box-shadow: 0 0 6px rgba(0,0,0,0.15); }
        th { background-color: #e9ecef; }
        footer { text-align: center; margin-top: 30px; font-size: 0.9em; color: #888; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Tender Search</h1>
        <form action="/" method="post">
            <input list="auth-list" type="text" name="organization_name" placeholder="Start typing an Authority Name..." value="{{ org_name or '' }}" autocomplete="off" required>
            <button type="submit">Search</button>
        </form>
        <datalist id="auth-list">
            {% for name in auth_names %}<option value="{{ name }}">{% endfor %}
        </datalist>

        {% if error_message %}
            <div class="alert alert-danger"><strong>Error:</strong> {{ error_message }}</div>
        {% endif %}

        {% if search_performed and not error_message and not results %}
            <div class="alert alert-info">No active tenders found for '{{ org_name }}'.</div>
        {% endif %}

        {% if results %}
            <div class="results-header">
                <h2>Found {{ results|length }} Tenders for '{{ org_name }}'</h2>
                <form action="/export" method="post">
                    <input type="hidden" name="organization_name" value="{{ org_name }}">
                    <button type="submit" class="export-button">Export to Excel</button>
                </form>
            </div>
            <table>
                <thead><tr><th>#</th><th>Tender Details</th><th>Location</th><th>Value</th><th>Due Date</th></tr></thead>
                <tbody>
                    {% for tender in results %}
                    <tr>
                        <td>{{ loop.index }}</td>
                        <td>{{ tender.requirement_workbrief }}</td>
                        <td>{{ tender.site_location }}</td>
                        <td>{{ tender.estimatedcost }}</td>
                        <td data-sort="{{ tender.sortable_due_date }}">{{ tender.tender_endsubmission_datetime }}</td>
                    </tr>
                    {% endfor %}
                </tbody>
            </table>
        {% endif %}
        <footer>-DEVLOPED BY ABHAY-</footer>
    </div>
</body>
</html>
"""

# ==============================================================================
# Step 5: Create the Flask Web Application with Corrected Logic
# ==============================================================================
app = Flask(__name__)

from flask import after_this_request

@app.before_request
def skip_ngrok_warning():
    @after_this_request
    def add_header(response):
        response.headers['ngrok-skip-browser-warning'] = 'true'
        return response

latest_search_results = {}
authority_names = load_auth_names()

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        org_name = request.form.get('organization_name')
        if not org_name:
            return render_template_string(HTML_TEMPLATE, error_message="Organization name cannot be empty.", auth_names=authority_names)

        api_response = search_tenders_by_org(org_name)

        if api_response.get('Success') == True:
            tender_list = api_response.get('Data', []) or [] # Ensure it's a list

            # Step 1: Add a sortable date key to each tender dictionary.
            for tender in tender_list:
                due = tender.get("tender_endsubmission_datetime")
                try:
                    # Convert 'DD-MM-YYYY' to a sortable 'YYYY-MM-DD' format.
                    formatted_due = datetime.strptime(due, "%d-%m-%Y").strftime("%Y-%m-%d")
                    tender["sortable_due_date"] = formatted_due
                except (ValueError, TypeError):
                    # For invalid or missing dates, use a string that will always sort last.
                    tender["sortable_due_date"] = "9999-12-31"

            # Step 2: Sort the list by the new 'sortable_due_date' key in ascending order.
            tender_list_sorted = sorted(tender_list, key=lambda t: t['sortable_due_date'])

            # Step 3: Store and use the newly sorted list for both the webpage and Excel export.
            latest_search_results[org_name] = tender_list_sorted
            return render_template_string(HTML_TEMPLATE, results=tender_list_sorted, org_name=org_name, search_performed=True, auth_names=authority_names)
        else:
            error_msg = api_response.get('Message', 'An unknown error occurred.')
            return render_template_string(HTML_TEMPLATE, error_message=error_msg, org_name=org_name, search_performed=True, auth_names=authority_names)

    return render_template_string(HTML_TEMPLATE, auth_names=authority_names)

@app.route('/export', methods=['POST'])
def export_to_excel():
    org_name = request.form.get('organization_name')
    if not org_name or org_name not in latest_search_results: return "No data to export.", 404
    tender_list = latest_search_results[org_name]
    if not tender_list: return "No tenders found to export.", 404

    # Using the correct keys from the JSON response
    df_data = [{
        'Tender ID': t.get('tender_id'),
        'Title': t.get('requirement_workbrief'),
        'Location': t.get('site_location'),
        'Estimated Cost': t.get('estimatedcost'),
        'Due Date': t.get('tender_endsubmission_datetime'),
        'Organization': t.get('organization_name')
    } for t in tender_list]

    df = pd.DataFrame(df_data)
    output = io.BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Tenders')
    output.seek(0)

    clean_org_name = "".join(c for c in org_name if c.isalnum() or c in (' ', '_')).rstrip()
    filename = f"Tenders_{clean_org_name}.xlsx"
    return send_file(output, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', as_attachment=True, download_name=filename)

# ==============================================================================
# Step 6: Start ngrok and Run the Flask App
# ==============================================================================
if NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTH_TOKEN_HERE":
    ngrok.kill()
    public_url = ngrok.connect(5000)
    print(f"🚀 Your web application is live! Open this URL in your browser:\n{public_url}")
    app.run(port=5000)
else:
    print("\n\nCould not start the server. Please set your NGROK_AUTH_TOKEN.")